In [1]:
using DataStructures
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using BenchmarkTools

## funzione hollowBall()

In [2]:
function hollowBall(r=1., R=1., angle1=pi, angle2=2*pi)
    function hollowBall0(shape=[24, 36, 3])
        V, CV = cuboidGrid(shape)
        V = [angle1/shape[1] 0 0; 0 angle2/shape[2] 0; 0 0 (R-r)/shape[3]]*V
        V = broadcast(+, V, [-(angle1)/2, -(angle2)/2, r])
        W = [V[:, k] for k=1:size(V, 2)]
        V = hcat(map(p->let
        (u, v, w)=p; [w*cos(u)*cos(v); w*cos(u)*sin(v); w*sin(u)] end, W)...)
        W, CW = simplifyCells(V, CV)
        return W, CW
    end
    return hollowBall0
end


hollowBall (generic function with 5 methods)

In [3]:
@code_warntype Lar.hollowBall()();

Variables
  #self#::LinearAlgebraicRepresentation.var"#hollowBall0#323"{Float64,Float64,Irrational{:π},Float64}

Body::Tuple{Array,Array{Array{Int64,1},1}}
1 ─ %1 = Base.vect(24, 36, 3)::Array{Int64,1}
│   %2 = (#self#)(%1)::Tuple{Array,Array{Array{Int64,1},1}}
└──      return %2


In [93]:
@btime W,CW = Lar.hollowBall()();

  19.369 ms (260857 allocations: 21.48 MiB)


In [5]:
@btime V, CV = Lar.hollowBall(1, 2, pi/2, pi/2)([6, 12, 4]);

  2.294 ms (27526 allocations: 2.44 MiB)


In [6]:
@btime V, CV = Lar.hollowBall(1, 2, pi/2, pi/2)([20, 30, 20]);

  107.435 ms (1316120 allocations: 101.41 MiB)


In [96]:
@benchmark Lar.hollowBall()()

BenchmarkTools.Trial: 229 samples with 1 evaluation.
 Range (min … max):  19.462 ms … 27.210 ms  ┊ GC (min … max): 0.00% … 15.05%
 Time  (median):     20.852 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   21.834 ms ±  2.254 ms  ┊ GC (mean ± σ):  9.17% ±  8.90%

  ██▅                                                          
  ███▄▃▂▁▁▁▁▂▂▂▁▂▁▁▂▂▁▂▁▁▂▁▂▁▁▁▁▁▁▁▁▃▅▄▄█▆▅▅▆▇▆▃▁▂▁▁▂▁▁▁▁▁▂▁▃ ▃
  19.5 ms         Histogram: frequency by time        26.1 ms <

 Memory estimate: 21.48 MiB, allocs estimate: 260855.

## Uso della macro @threads 

In [72]:
using Base.Threads
Threads.nthreads() = 2  #setto il numero di threads pari a 2
nthreads()

2

#### Per settare il numero di threads pari a 2 ho dovuto per forza eseguire (come scritto nella guida di julia) i seguenti passaggi :  export JULIA_NUM_THREADS=2  , set JULIA_NUM_THREADS=2 , il numero di threads va settato in base alla macchina detenuta , se non lo si fa l'uso dei threads è nullo .

Esempio effetti dei @threads

In [79]:
@time for i=1:10; rand(1000000); end

  0.061409 seconds (20 allocations: 76.295 MiB, 17.17% gc time)


In [80]:
@time @threads for i=1:10; rand(1000000); end

  0.035389 seconds (29.30 k allocations: 77.850 MiB, 9.50% gc time)


----------------------------------------------------------------------------------------------------------------
## Refactoring codice

In [100]:
function hollowBall1(r=1., R=1., angle1=pi, angle2=2*pi)
    function hollowBall01(shape=[24, 36, 3])
        V, CV = Lar.cuboidGrid(shape)
        V = [angle1/shape[1] 0 0; 0 angle2/shape[2] 0; 0 0 (R-r)/shape[3]]*V
        V = broadcast(+, V, [-(angle1)/2, -(angle2)/2, r])
        n = size(V,2)
        W = Array{Array{Float64,1}}(undef, n)
        
        @inbounds @threads for k=1:n
         W[k] = V[:,k] 
    end
        V = hcat(map(p->let
        (u, v, w)=p; [w*cos(u)*cos(v); w*cos(u)*sin(v); w*sin(u)] end, W)...)
        W, CW = Lar.simplifyCells(V, CV)
        return W, CW
    end
    return hollowBall01
end


hollowBall1 (generic function with 5 methods)

In [101]:
@btime W,CW = hollowBall1()();

  13.441 ms (180538 allocations: 14.89 MiB)


In [27]:
@btime V, CV = hollowBall1(1, 2, pi/2, pi/2)([6, 12, 4]);

  2.296 ms (27527 allocations: 2.44 MiB)


In [99]:
@btime V, CV = hollowBall1(1, 2, pi/2, pi/2)([20, 30, 20]);

  58.984 ms (747444 allocations: 56.86 MiB)


In [97]:
@benchmark hollowBall1()()

BenchmarkTools.Trial: 221 samples with 1 evaluation.
 Range (min … max):  19.564 ms … 35.528 ms  ┊ GC (min … max): 0.00% … 19.27%
 Time  (median):     23.661 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   22.645 ms ±  3.173 ms  ┊ GC (mean ± σ):  9.56% ±  9.32%

  █▇▃▁             ▁▅▆▄▂▂                                      
  ████▆▄▄▁▁▄▁▁▁▁▄▁████████▁▄▁▁▁▄▄▄▄▁▁▁▁▁▁▁▁▁▄▁▆▁▁▁▁▁▁▁▄▆▁▁▁▁▄ ▆
  19.6 ms      Histogram: log(frequency) by time      34.3 ms <

 Memory estimate: 21.53 MiB, allocs estimate: 264048.

Sorprendente vedere quando si passano dei parametri come tutti i valori/tempistiche cambino e anche di molto .